In [2]:
import tensorflow as tf
import sys
sys.path.append('../lib')
from table_detection import TableNet
from utils import *
import cv2
from shutil import copyfile

In [3]:
import os

In [4]:
model = TableNet.build()
latest = tf.train.latest_checkpoint('models/2020_09_23')
model.load_weights(latest)

In [5]:
img_height, img_width = 256, 256
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width]), img.shape

def normalize(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image

def get_image_tensor(file):
    decoded_img, shape = decode_img(tf.io.read_file(file))
    return normalize(decoded_img), shape
def save_pred(filename,pred):
    plt.imsave(filename,np.repeat(255*tf.keras.preprocessing.image.array_to_img(column_mask),3).reshape(256,256,3))

In [6]:
for file in [os.path.join('../data/test/',x) for x in os.listdir('../data/test/') if x.split('.')[-1]=='jpg']:
    img_tensor,shape = get_image_tensor(file)
    image = tf.expand_dims(img_tensor,axis=0)
    table_mask, column_mask = get_masks(model,image)
    plt.imsave(os.path.join('../results/test_table_mask',file.split('/')[-1]),tf.keras.preprocessing.image.array_to_img(table_mask),cmap='gray')
    plt.imsave(os.path.join('../results/test_column_mask',file.split('/')[-1]),tf.keras.preprocessing.image.array_to_img(column_mask),cmap='gray')
    copyfile(file, os.path.join('../results/test_image',file.split('/')[-1]))

1/1 [==============================] - 0s 68ms/sample
